In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= '0'
#!pip install import_ipynb
import import_ipynb

if os.getcwd() != '/home/mskang/hyeokjong/cancer/2019/custom_model':
    path = '/home/mskang/hyeokjong/cancer/2019/custom_model'
    os.chdir('/home/mskang/hyeokjong/cancer/2019/custom_model')
    print('directory changed !' )
    print( 'current directoy is', str(os.getcwd()) )
else:
    print( 'current directoy is', str(os.getcwd()) )
from data_224_128 import*
import os
import numpy as np
import torch
import torchvision
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import cv2
import torch.nn as nn
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import copy
import time
import csv
from tqdm import tqdm
from torch.utils.data.dataset import random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

current directoy is /home/mskang/hyeokjong/cancer/2019/custom_model
Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [3]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Interpolation(nn.Module):
    def __init__(self, out_size):
        super().__init__()
        self.size = out_size
        
    def forward(self, x):
        return torch.nn.functional.interpolate(x, size = self.size, mode = 'bilinear', align_corners = True)

In [4]:
class residual_block(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.relu = nn.ReLU()

        self.resi =  nn.Sequential( nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels = out_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel) )
        
        self.identity = nn.Sequential( nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU())
    def forward(self, x):
        resi = self.resi(x)
        identity = self.identity(x)
        out = resi + identity
        out = self.relu(out)
        return out

In [5]:
class residual_block_con_connect(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.relu = nn.ReLU()

        self.resi =  nn.Sequential( nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels = out_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel) )
        

    def forward(self, x):
        resi = self.resi(x)
        out = resi +x
        out = self.relu(out)
        return out

In [6]:
class att_resi(nn.Module):
    
    def __init__(self, in_channel, out_channel):
        super().__init__()

        self.relu = nn.ReLU()
        self.s = nn.Sigmoid()
        self.resi =  nn.Sequential( nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU(),
                                    nn.Conv2d(in_channels = out_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel) )
        self.conv_connect = nn.Sequential( nn.Conv2d(in_channels = in_channel, out_channels = out_channel, kernel_size = 3, stride = 1, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU() )
        self.conv = nn.Sequential( nn.Conv2d(in_channels = out_channel, out_channels = out_channel, kernel_size = 3, stride = 2, padding = 1, bias = False),
                                    nn.BatchNorm2d(num_features = out_channel),
                                    nn.ReLU() )

        

    def forward(self, x):
        resi = self.resi(x)
        out = resi + self.conv_connect(x)
        out = self.relu(out)
        out = self.conv(out)
        out = self.s(out)
        return out

In [7]:
class attention_naive(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.maxpool = nn.MaxPool2d(kernel_size = 2)
        self.sigmoid = nn.Sigmoid()
        self.interpolation128 = Interpolation(128)
        self.interpolation64 = Interpolation(64)
        self.interpolation32 = Interpolation(32)
        
        
        self.c1 = nn.Sequential( nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 1, stride = 1, padding = 0, bias = False),
                                   nn.BatchNorm2d(num_features = 16),   
                                   nn.ReLU())
        self.c2 = nn.Sequential( nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 1, stride = 1, padding = 0, bias = False),
                                   nn.BatchNorm2d(num_features = 32),   
                                   nn.ReLU())
        self.c3 = nn.Sequential( nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 1, stride = 1, padding = 0, bias = False),
                                   nn.BatchNorm2d(num_features = 64),   
                                   nn.ReLU())
        
        self.conv1 = self.make_residual_block_att(1, 3, 16) 
        self.conv2 = self.make_residual_block_att(1, 32, 32) 
        self.conv3 = self.make_residual_block_att(1, 64, 64) 
        
        
        
        self.att1 = self.make_residual_block_att(1, 3, 16) 
        self.att2 = self.make_residual_block_att(1, 16, 32) 
        self.att3 = self.make_residual_block_att(1, 32, 64)  
        

        
        
        
        
        self.resi4_1 = self.make_residual_block(1, 128, 256) 
        self.resi4_2 = self.make_residual_block(1, 256, 256) 
        
        self.resi5_1 = self.make_residual_block(1, 256, 256) 
        self.resi5_2 = self.make_residual_block(1, 256, 256) 
        self.resi5_3 = self.make_residual_block(1, 256, 512) 
        
        self.resi6_1 = self.make_residual_block(1, 512, 512) 
        self.resi6_2 = self.make_residual_block(1, 512, 512) 
        self.resi6_3 = self.make_residual_block(1, 512, 1024) 

        self.Flatten = Flatten()
        self.classifier = nn.Sequential( Flatten(),
                                         nn.Linear(1024*4*4,1000),
                                         nn.ReLU(),
                                         nn.Linear(1000, 8))
        
        
        
        
        
        
        
        
        
    def make_residual_block(self, num_modules, in_channel, out_channel):
        layers = list()
        for i in range(num_modules):
            if in_channel != out_channel:
                layers.append(residual_block(in_channel, out_channel))
            elif in_channel == out_channel:
                layers.append(residual_block_con_connect(in_channel, out_channel))
        return nn.Sequential( *layers )

    
    def make_residual_block_att(self, num_modules, in_channel, out_channel):
        layers = list()
        layers.append(att_resi(in_channel, out_channel))
        
        return nn.Sequential( *layers )

    
    
    def forward(self, raw):   # 3 256
        
        raw1 = self.interpolation128(raw) # 3 128
        raw2 = self.interpolation64(raw1) # 3 64
        raw3 = self.interpolation32(raw2) # 3 32
        
        y = self.conv1(raw) # 16 128
        
        x = self.att1(raw) + 1 # 16 128
        x = x*self.c1(raw1)    # 16 128
        
        y = torch.cat( (y,x), dim = 1 )

        y = self.conv2(y)
        
        x = self.att2(x) + 1
        x = x*self.c2(raw2)
        
        y = torch.cat( (y,x), dim = 1 )
        y = self.conv3(y)
        
        x = self.att3(x) + 1
        x = x*self.c3(raw3)
        
        y = torch.cat( (y,x), dim = 1 )

        x = self.resi4_1(y)
        x = self.resi4_2(x)
        x = self.maxpool(x)
        
        x = self.resi5_1(x)
        x = self.resi5_2(x)
        x = self.resi5_3(x)
        x = self.maxpool(x)        
        
        x = self.resi6_1(x)
        x = self.resi6_2(x)
        x = self.resi6_3(x)
        x = self.maxpool(x)

        
        x = self.classifier(x)

        return x

In [11]:
model = attention_naive().to(device)
path_model = '/home/mskang/hyeokjong/cancer/2019/custom_model/A_final_3.pt'
model.load_state_dict(torch.load(path_model))

<All keys matched successfully>

In [14]:
def metric_function(output, target):
    _, argmax = torch.max(output, dim = 1)
    corrects = (argmax == target).sum()
    return corrects 

running_loss = 0.0
running_metric = 0.0
loss_func = nn.CrossEntropyLoss(reduction = 'sum')
with torch.no_grad():       
    running_loss = 0.0
    running_metric = 0.0
        
    len_data_test = len(test_dl.dataset)
            
    for inputs, targets in test_dl:
        inputs , targets = inputs.to(device) , targets.to(device)
        outputs = model(inputs)
                
        loss_batch = loss_func(outputs, targets)
        metric_batch = metric_function(outputs, targets)
                
        running_loss += loss_batch.item()
        running_metric += metric_batch
    test_loss = running_loss / len_data_test
    test_metric = running_metric / len_data_test
print('test loss: %.6f, test accuracy: %.2f' %(test_loss, 100*test_metric))

test loss: 0.041877, test accuracy: 98.83
